In [176]:
#Import Dependencies#
import os
import pandas as pd
from glob import glob
import requests
import pandas as pd
import geojson

In [6]:
#Using glob module to store csv rows#
filenames = glob('new_york/*.csv')
dataframe_list = [pd.read_csv(f) for f in filenames]
dataframe_list.append(pd.read_csv("new_york_current.csv", low_memory=False))

In [7]:
#Concatenate all stored dataframes created above#
all_data_df = pd.concat(dataframe_list, sort=False)

In [8]:
#Display Data#
all_data_df.head()

,room_id,host_id,room_type,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,...,latitude,longitude,last_modified,survey_id,country,city,bathrooms,name,property_type,location
0,105.0,500.0,Private room,Manhattan,Hell's Kitchen,39.0,5.0,7.0,1.0,167.0,...,40.762099,-73.996022,2014-05-12 08:56:52.946,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2515.0,2758.0,Private room,Manhattan,Harlem,95.0,4.5,2.0,1.0,55.0,...,40.799205,-73.953676,2014-05-12 18:11:09.122,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2993.0,3352.0,Entire home/apt,Manhattan,Lower East Side,0.0,NaN,2.0,1.0,168.0,...,40.723593,-73.991188,2014-05-12 15:16:46.188,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3046.0,2758.0,Entire home/apt,Manhattan,Harlem,10.0,4.5,6.0,1.0,298.0,...,40.813498,-73.945873,2014-05-12 19:47:22.038,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3101.0,2758.0,Private room,Manhattan,Harlem,11.0,4.5,2.0,1.0,140.0,...,40.813628,-73.946299,2014-05-12 00:57:13.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#List Column Names#
list(all_data_df.columns.values)

['room_id',
 'host_id',
 'room_type',
 'borough',
 'neighborhood',
 'reviews',
 'overall_satisfaction',
 'accommodates',
 'bedrooms',
 'price',
 'minstay',
 'latitude',
 'longitude',
 'last_modified',
 'survey_id',
 'country',
 'city',
 'bathrooms',
 'name',
 'property_type',
 'location']

In [10]:
#Keeping only the relevant columns#
all_data_df = all_data_df.loc[:,["last_modified", "price", "neighborhood", "borough", "latitude", "longitude", "overall_satisfaction", "room_type"]]
all_data_df.head()

,last_modified,price,neighborhood,borough,latitude,longitude,overall_satisfaction,room_type
0,2014-05-12 08:56:52.946,167.0,Hell's Kitchen,Manhattan,40.762099,-73.996022,5.0,Private room
1,2014-05-12 18:11:09.122,55.0,Harlem,Manhattan,40.799205,-73.953676,4.5,Private room
2,2014-05-12 15:16:46.188,168.0,Lower East Side,Manhattan,40.723593,-73.991188,NaN,Entire home/apt
3,2014-05-12 19:47:22.038,298.0,Harlem,Manhattan,40.813498,-73.945873,4.5,Entire home/apt
4,2014-05-12 00:57:13.100,140.0,Harlem,Manhattan,40.813628,-73.946299,4.5,Private room


In [11]:
#Cleaning Up Column Names#
all_data_df = all_data_df.rename(index=str, columns={"last_modified": "Date", "overall_satisfaction": "Review", "room_type": "Room Type"})
all_data_df = all_data_df.rename(str.title, axis='columns')
all_data_df

,Date,Price,Neighborhood,Borough,Latitude,Longitude,Review,Room Type
0,2014-05-12 08:56:52.946,167.0,Hell's Kitchen,Manhattan,40.762099,-73.996022,5.0,Private room
1,2014-05-12 18:11:09.122,55.0,Harlem,Manhattan,40.799205,-73.953676,4.5,Private room
2,2014-05-12 15:16:46.188,168.0,Lower East Side,Manhattan,40.723593,-73.991188,NaN,Entire home/apt
3,2014-05-12 19:47:22.038,298.0,Harlem,Manhattan,40.813498,-73.945873,4.5,Entire home/apt
4,2014-05-12 00:57:13.100,140.0,Harlem,Manhattan,40.813628,-73.946299,4.5,Private room
5,2014-05-12 01:03:09.110,212.0,Harlem,Manhattan,40.814100,-73.945891,5.0,Private room
6,2014-05-12 05:13:53.328,112.0,Astoria,Queens,40.759736,-73.906151,5.0,Private room
7,2014-05-12 08:45:14.766,100.0,Bedford-Stuyvesant,Brooklyn,40.678300,-73.944627,4.5,Entire home/apt
8,2014-05-12 21:15:38.208,112.0,Clinton Hill,Brooklyn,40.685138,-73.959757,NaN,Private room
9,2014-05-12 19:01:16.331,202.0,Crown Heights,Brooklyn,40.674376,-73.933990,4.0,Entire home/apt


In [12]:
#Lookup all null value types and drop extraneous rows#
null_columns=all_data_df.columns[all_data_df.isnull().any()]
all_data_df[null_columns].isnull().sum()
num_rows= 2092268
num_rows_null= 999519
index_where_null = num_rows - num_rows_null
all_data_df = all_data_df.drop(all_data_df.index[index_where_null:(index_where_null+num_rows_null)])
all_data_df
null_columns=all_data_df.columns[all_data_df.isnull().any()]
all_data_df[null_columns].isnull().sum()

Review       267256
Room Type       318
dtype: int64

In [13]:
#Getting Date column to just show year#
rows = [];
for row in all_data_df.loc[:,"Date"]:
    row = row[0:4]
    rows.append(row)
all_data_df["Date"] = rows
all_data_df.head()

C:\Users\romys\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Date,Price,Neighborhood,Borough,Latitude,Longitude,Review,Room Type
0,2014,167.0,Hell's Kitchen,Manhattan,40.762099,-73.996022,5.0,Private room
1,2014,55.0,Harlem,Manhattan,40.799205,-73.953676,4.5,Private room
2,2014,168.0,Lower East Side,Manhattan,40.723593,-73.991188,NaN,Entire home/apt
3,2014,298.0,Harlem,Manhattan,40.813498,-73.945873,4.5,Entire home/apt
4,2014,140.0,Harlem,Manhattan,40.813628,-73.946299,4.5,Private room


In [14]:
#write all data to csv#
all_data_df.to_csv("output_csv/all_historical_data.csv")

PermissionError: [Errno 13] Permission denied: 'output_csv/all_historical_data.csv'

In [15]:
#Create average price dataframe#
pd.to_numeric(all_data_df["Price"])
price_summary_df = pd.DataFrame(all_data_df.groupby("Date")["Price"].mean())
price_summary_df = price_summary_df.rename(index=str,columns={"Price":"Average Price"})
price_summary_df

,Average Price
Date,
2014,210.424649
2015,198.485984
2016,147.455851
2017,138.821924
2018,152.330031


In [16]:
#write price summary data to csv#
price_summary_df.to_csv("output_csv/historical_price_data.csv")

In [143]:
#Create borough average price summary dataframe#
neighborhood_summary_df = pd.DataFrame(all_data_df.groupby(["Borough", "Neighborhood","Date"])["Price"].mean())
neighborhood_summary_df.reset_index(inplace=True)
neighborhood_summary_df
neighborhood_summary_df['Neighborhood'].str.contains('Allerton').any()
choro_df = pd.DataFrame(neighborhood_summary_df.groupby("Neighborhood")["Price"].mean()).reset_index()
choro_df
data_df = neighborhood_summary_df[neighborhood_summary_df.loc[:,"Neighborhood"]=="Allerton"][["Date", "Price"]]
for index, row in data_df.iterrows():
    print(row['Date'], row['Price'])

2014 68.82608695652173
2015 69.58252427184466
2016 64.5
2017 66.33557046979865
2018 89.28947368421052


In [184]:
#write summary data to csv#
neighborhood_summary_df.to_csv("output_csv/neighborhood_data.csv")

In [153]:
borough_summary_df = pd.DataFrame(neighborhood_summary_df.groupby(["Borough", "Date"])["Price"].mean())
borough_summary_df

Price
Borough       Date            
Bronx         2014   92.522855
              2015  102.378403
              2016   80.573626
              2017  100.178549
              2018   87.109599
Brooklyn      2014  146.953098
              2015  149.372979
              2016  122.980335
              2017  123.080343
              2018  131.196663
Manhattan     2014  265.930938
              2015  257.008236
              2016  189.255071
              2017  182.802989
              2018  208.564560
Queens        2014  118.848403
              2015  116.530135
              2016  118.249138
              2017  100.176765
              2018  114.033138
Staten Island 2014  311.877790
              2015  204.587774
              2016  148.770574
              2017  129.653919
              2018  131.188337

In [185]:
#write sumary data to csv#
borough_summary_df.to_csv("output_csv/borough_historical_data.csv")

In [177]:
#Create room_type average price summary dataframe#
neighborhood_roomtype_summary_df = pd.DataFrame(all_data_df.groupby(["Borough","Room Type","Date"])["Price"].mean())
neighborhood_roomtype_summary_df

Price
Borough       Room Type       Date            
Bronx         Entire home/apt 2014  144.738589
                              2015  173.655589
                              2016  122.754171
                              2017  124.229790
                              2018  117.576433
              Private room    2014   75.399247
                              2015   72.522265
                              2016   65.479079
                              2017   59.242754
                              2018   67.960139
              Shared room     2014   59.125000
                              2015   56.338710
                              2016   57.516269
                              2017   53.776699
                              2018   80.297297
Brooklyn      Entire home/apt 2014  207.701774
                              2015  210.963118
                              2016  174.499730
                              2017  168.320552
                              2018  175.497537
              Private room    2014   97.929289
                              2015   91.986838
                              2016   75.877780
                              2017   70.552648
                              2018   74.969941
              Shared room     2014   67.374175
                              2015   67.172551
                              2016   54.499601
                              2017   53.087460
                              2018   53.670157
...                                        ...
Queens        Entire home/apt 2014  168.882277
                              2015  163.395042
                              2016  144.450070
                              2017  140.020939
                              2018  145.179173
              Private room    2014   87.555216
                              2015   83.958520
                              2016   73.279130
                              2017   64.697249
                              2018   76.156270
              Shared room     2014  111.887324
                              2015   86.772118
                              2016   61.812045
                              2017   49.897991
                              2018   65.804878
Staten Island Entire home/apt 2014  354.216346
                              2015  309.997768
                              2016  196.090539
                              2017  183.683444
                              2018  174.538462
              Private room    2014   76.890805
                              2015   72.164725
                              2016   69.644529
                              2017   62.221344
                              2018   59.779070
              Shared room     2014   41.000000
                              2015   54.666667
                              2016   40.925926
                              2017   18.727273
                              2018   52.600000

[75 rows x 1 columns]

In [186]:
#write room type data to csv#
neighborhood_roomtype_summary_df.to_csv("output_csv/roomtype_historical_data.csv")

In [175]:
import json

with open('neighborhoods.geojson') as f:
    data = json.load(f)

count_add = 0
count_del = 0
indices = [];
count = 0

print(len(data['features']))

for feature in data['features']:
    change_property = feature['properties']
    neigh_name = feature['properties']['neighbourhood']
    data_df = neighborhood_summary_df[neighborhood_summary_df.loc[:,"Neighborhood"]== neigh_name][["Date", "Price"]]

    if neighborhood_summary_df["Neighborhood"].str.contains(neigh_name).any() and (len(data_df) == 5):
        count_add = count_add + 1
        
        for index, row in data_df.iterrows():
            change_property[row['Date']] = round(row['Price'],2)
        
    else:
        count_del = count_del + 1
        indices.append(count)
        #data["features"].remove(feature)
    count = count + 1

print(len(data['features']))  
print(count_add)
print(count_del)
print(count)
print(indices)
print(len(indices))

count = 0
for ind in indices:
    data["features"].remove(data["features"][ind-count])
    count = count + 1

print(len(data['features'])) 
data["features"][0]["properties"]

233
233
200
33
233
[5, 9, 14, 20, 25, 28, 31, 39, 42, 49, 67, 72, 100, 104, 106, 112, 115, 120, 121, 124, 125, 135, 153, 159, 170, 173, 179, 181, 191, 192, 194, 224, 232]
33
200


{'neighbourhood': 'Bayswater',
 'neighbourhood_group': 'Queens',
 '2014': 106.67,
 '2015': 79.81,
 '2016': 76.97,
 '2017': 76.78,
 '2018': 74.11}

In [180]:
with open('neighborhoods_price.geojson', 'w') as outfile:
     geojson.dump(data, outfile)

In [131]:
len(choro_df.loc[:,"Neighborhood"])

247

In [123]:
name_list = []
length = len(response["features"])
for i in range(0,length):
    change = response["features"][i]["properties"]
    name_list.append(change["NTAName"])
    if choro_df["Neighborhood"].str.contains(change["NTAName"]).any() & (not choro_df[choro_df.loc[:,"Neighborhood"]==change["NTAName"]]["Price"].empty) :
        change["Average Price"] = choro_df[choro_df.loc[:,"Neighborhood"]==change["NTAName"]]["Price"].values[0]
print(sorted(name_list))

['Airport', 'Allerton-Pelham Gardens', "Annadale-Huguenot-Prince's Bay-Eltingville", 'Arden Heights', 'Astoria', 'Auburndale', 'Baisley Park', 'Bath Beach', 'Battery Park City-Lower Manhattan', 'Bay Ridge', 'Bayside-Bayside Hills', 'Bedford', 'Bedford Park-Fordham North', 'Bellerose', 'Belmont', 'Bensonhurst East', 'Bensonhurst West', 'Borough Park', 'Breezy Point-Belle Harbor-Rockaway Park-Broad Channel', 'Briarwood-Jamaica Hills', 'Brighton Beach', 'Bronxdale', 'Brooklyn Heights-Cobble Hill', 'Brownsville', 'Bushwick North', 'Bushwick South', 'Cambria Heights', 'Canarsie', 'Carroll Gardens-Columbia Street-Red Hook', 'Central Harlem North-Polo Grounds', 'Central Harlem South', 'Charleston-Richmond Valley-Tottenville', 'Chinatown', 'Claremont-Bathgate', 'Clinton', 'Clinton Hill', 'Co-op City', 'College Point', 'Corona', 'Crotona Park East', 'Crown Heights North', 'Crown Heights South', 'Cypress Hills-City Line', 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill', 'Douglas Manor-Douglas

C:\Users\romys\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [118]:
print(length)
len(index_del)


195


126